In [2]:
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv", header = None)
col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ,]
for i in range(1,26):
    col.append("x"+str(i))
data.columns = col
data = data.astype({"treatment":'bool'}, copy=False)
data.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,True,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,1,1,1,1,0,0,0,0,0,0
1,False,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,1,1,1,1,0,0,0,0,0,0
2,False,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,1,0,1,1,0,0,0,0,0,0
3,False,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,1,0,1,1,0,0,0,0,0,0
4,False,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,1,1,1,1,0,0,0,0,0,0


In [4]:
model=CausalModel(
        data = data,
        treatment='treatment',
        outcome='y_factual',
        common_causes=["x"+str(i) for  i in range(1,26)]
        )

## Propensity Score Stratification

It divides the dataset into strata (bins) based on propensity scores, ensuring treatment and control groups within each stratum are comparable. 

$$
Stratification\ Weights (Strata\ 'n') = \frac{Number\ of\ element\ in\ strata\ 'n'}{Total\ number\ of\ elements} 
$$
$
Treatment\ effect = (ATE\ of\ each\ strata).(Weights\ associated\ to\ each\ strata) 
$
$
where, . = Dot\ product
$

In [15]:
class PropensityScoreEstimator:
    def __init__(self, treatment_column, outcome_column, confounders_columns):
        self.treatment_col = treatment_column
        self.outcome_col = outcome_column
        self.confounder_cols = confounders_columns
        self.model = LogisticRegression()
        self.propensity_scores = None

    def fit(self, data):
        X = data[self.confounder_cols]
        # treatment
        T = data[self.treatment_col]

        self.model.fit(X, T)
        self.propensity_scores = self.model.predict_proba(X)[:, 1]
        data['propensity_score'] = self.propensity_scores

        return data

    def match(self, data):
        """
        Matching based on propensity scores
        """
        treated = data[data[self.treatment_col]==1]
        print(f'Treated shape: {treated.shape}')
        control = data[data[self.treatment_col]==0]
        print(f'Control shape: {control.shape}')
        matched_control = []
        for _, treated_row in treated.iterrows():
            # taking control row with closest propensity score to treated row
            control_row = control.loc[(control['propensity_score'] - treated_row['propensity_score']).abs().idxmin()]
            matched_control.append(control_row)

        matched_control = pd.DataFrame(matched_control)
        matched_data = pd.concat([treated, matched_control])
        return matched_data

    def estimate_effect(self, data):
        treated = data[data[self.treatment_col]==1]
        control = data[data[self.treatment_col]==0]

        treated_mean = treated[self.outcome_col].mean()
        control_mean = control[self.outcome_col].mean()
        return treated_mean - control_mean

    def stratify(self, data, n_strata=5):
        # divide the dataset into n parts almost equally
        data['stratum'] = pd.qcut(data['propensity_score'], q=n_strata, labels=False)

        treatment_effects = []
        weights = []

        for stratum in range(n_strata):
            stratum_data = data[data['stratum']==stratum]
            treated = stratum_data[stratum_data[self.treatment_col] == 1]
            control = stratum_data[stratum_data[self.treatment_col] == 0]

            if treated.empty or control.empty:
                continue 

            treated_mean = treated[self.outcome_col].mean()
            control_mean = control[self.outcome_col].mean()

            treatment_effect = treated_mean - control_mean 
            treatment_effects.append(treatment_effect)
            weights.append(len(stratum_data)/len(data))

        print(treatment_effects)
        print(weights)

        # weighted average treatment effect
        weights = np.array(weights)
        overall_effect = np.dot(weights, treatment_effects)
        return overall_effect 

In [16]:
estimator = PropensityScoreEstimator(treatment_column='treatment', outcome_column='y_factual', confounders_columns=["x"+str(i) for  i in range(1,26)])
estimator

In [17]:
df = estimator.fit(data)
df.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x18,x19,x20,x21,x22,x23,x24,x25,propensity_score,stratum
0,True,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,1,1,0,0,0,0,0,0,0.283505,3
1,False,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,1,1,0,0,0,0,0,0,0.068051,0
2,False,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,1,1,0,0,0,0,0,0,0.107717,1
3,False,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,1,1,0,0,0,0,0,0,0.129523,2
4,False,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,1,1,0,0,0,0,0,0,0.092541,1


In [18]:
effect_stratification = estimator.stratify(df)
effect_stratification 

[4.111720436173342, 4.3677116625044565, 4.053667877374529, 4.035469062740869, 3.6475109532226586]
[0.20080321285140562, 0.1994645247657296, 0.1994645247657296, 0.1994645247657296, 0.20080321285140562]
